In [1]:
from astropy.coordinates import EarthLocation, AltAz, ITRS
from scipy import interpolate
import astropy.units as u
import numpy as np
from astropy.time import Time

In [2]:
from lofarantpos.db import LofarAntennaDatabase

In [3]:
from pyproj import Transformer

In [4]:
db = LofarAntennaDatabase()

In [5]:
loc_rs305 = EarthLocation(*(db.phase_centres["RS305LBA"]*u.m))
loc_de605 = EarthLocation(*(db.phase_centres["DE605LBA"]*u.m))

In [6]:
dummytime = Time.now()

In [7]:
wgs84_to_rds = Transformer.from_crs(4326, 28992)
rds_to_wgs84 = Transformer.from_crs(28992, 4326)

In [8]:
def distance_from_height(obs_altaz, height=100*u.km):
    """From an AltAz pointing, find the distance to where it crosses a plane 100 km above Earth"""
    try_distances = np.linspace(50, 1000) * u.km
    try_altaz = AltAz(location=obs_altaz.location, obstime=dummytime, az=obs_altaz.az, alt=obs_altaz.alt, distance=try_distances)
    try_heights = EarthLocation(*(try_altaz.transform_to(ITRS(obstime=dummytime)).cartesian.xyz)).height
    height_to_distance = interpolate.interp1d(try_heights.to(u.km), try_distances.to(u.km))
    return height_to_distance(height) * u.km

In [9]:
def earthloc_at_height(obs_altaz, height=100*u.km):
    """From an AltAz pointing, find the EarthLocation at which point the line of sight is exactly at a given height (100km)"""
    distance = distance_from_height(obs_altaz, height)
    intersection_altaz = AltAz(location=obs_altaz.location, obstime=dummytime, az=obs_altaz.az, alt=obs_altaz.alt, distance=distance)
    return EarthLocation(*(intersection_altaz.transform_to(ITRS(obstime=dummytime)).cartesian.xyz))

In [10]:
def draw_fov(
    obsloc: EarthLocation, alt0, az0, fwhm=13 * u.deg, height=100 * u.km, num_segments=31
):
    """
    Make well-known-text (WKT) representation of beam at a certain height
    Coordinate system is rijksdriehoeksstelsel valid in NL
    """
    nodes = []
    for angle_offset in np.linspace(0, 360, num_segments) * u.deg:
        alt = alt0 + 0.5 * fwhm * np.cos(angle_offset)
        az = az0 + 0.5 * fwhm * np.sin(angle_offset)
        obs_altaz = AltAz(location=obsloc, obstime=dummytime, alt=alt, az=az)
        loc = earthloc_at_height(obs_altaz, height=height)
        node = wgs84_to_rds.transform(loc.lat.degree, loc.lon.degree, loc.height.value)[:2]
        nodes += [node]
    nodes += [nodes[0]]
    return "POLYGON((" +  ", ".join([f"{node[0]:.2f} {node[1]:.2f}" for node in nodes]) + "))"

In [37]:
alt_rs305 = 30 * u.deg

In [38]:
for az0 in np.array([200, 210, 220]) * u.deg:
    print(draw_fov(loc_rs305, alt_rs305, az0, fwhm=5.16*u.deg, height=100*u.km))

POLYGON((199329.67 403690.76, 197887.38 403856.71, 196288.87 403428.15, 194584.46 402411.98, 192831.30 400832.48, 191093.28 398732.18, 189437.86 396167.17, 187938.13 393218.78, 186668.90 389992.09, 185702.41 386616.02, 185102.92 383240.90, 184920.92 380034.63, 185186.71 377173.71, 185899.16 374814.90, 187030.22 373100.72, 188522.13 372137.51, 190291.37 371983.92, 192235.97 372644.82, 194245.10 374071.62, 196209.22 376168.91, 198031.30 378813.54, 199627.06 381847.50, 200933.46 385107.19, 201909.05 388434.46, 202531.91 391683.14, 202796.69 394724.39, 202711.27 397449.13, 202294.59 399771.69, 201571.04 401619.31, 200570.89 402937.76, 199329.67 403690.76, 199329.67 403690.76))
POLYGON((175238.60 414367.42, 173847.04 414781.22, 172198.41 414636.66, 170343.46 413931.79, 168342.69 412680.62, 166266.41 410913.92, 164190.79 408675.21, 162201.95 406031.93, 160391.78 403074.56, 158853.83 399917.52, 157677.47 396697.71, 156941.57 393571.73, 156706.63 390708.10, 156998.73 388261.44, 157815.01 38637

In [34]:
de605_pointings = []
for rs305_az in np.array([200, 210, 220]) * u.deg:
    rs305_altaz_far = AltAz(location=loc_rs305, alt=alt_rs305, az=rs305_az)
    rs305_altaz_near = AltAz(
        location=loc_rs305,
        alt=alt_rs305,
        az=rs305_az,
        distance=distance_from_height(rs305_altaz_far, 100 * u.km),
        obstime=dummytime,
    )
    de605_altaz_near = rs305_altaz_near.transform_to(
        AltAz(location=loc_de605, obstime=dummytime)
    )
    de605_pointings.append(de605_altaz_near)

In [39]:
de605_pointings

[<AltAz Coordinate (obstime=2020-12-07 21:31:46.840659, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (333.64971411, 53.07061514, 124377.461752)>,
 <AltAz Coordinate (obstime=2020-12-07 21:31:46.840659, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (322.90088284, 44.90473886, 140371.98250258)>,
 <AltAz Coordinate (obstime=2020-12-07 21:31:46.840659, location=(4005681.742, 450968.282, 4926457.67) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
     (318.76689597, 37.6535578, 161398.25362562)>]

In [40]:
for de605_pointing in de605_pointings:
    print(
        draw_fov(
            loc_de605,
            de605_pointing.alt,
            de605_pointing.az,
            fwhm=6.46 * u.deg,
            height=100 * u.km,
        )
    )

POLYGON((198070.44 381718.68, 198676.58 382219.44, 199111.52 383005.94, 199362.30 384053.61, 199419.55 385327.68, 199277.46 386783.14, 198934.43 388365.10, 198394.17 390009.69, 197667.39 391645.83, 196773.53 393197.84, 195742.14 394589.10, 194613.72 395746.27, 193438.66 396605.33, 192275.08 397116.88, 191185.47 397250.01, 190231.53 396995.96, 189468.80 396369.27, 188941.47 395406.56, 188678.42 394163.08, 188690.98 392707.58, 188972.91 391116.11, 189501.59 389466.93, 190241.07 387835.71, 191146.34 386291.15, 192167.26 384893.16, 193252.31 383691.90, 194351.81 382727.64, 195420.21 382031.09, 196417.55 381623.78, 197310.13 381518.37, 198070.44 381718.68, 198070.44 381718.68))
POLYGON((174095.41 392670.17, 174779.01 393466.83, 175185.81 394569.29, 175301.90 395944.40, 175119.79 397548.46, 174638.80 399326.82, 173866.18 401213.84, 172818.76 403133.73, 171524.92 405002.42, 170026.38 406730.70, 168379.32 408228.87, 166654.00 409412.77, 164932.62 410210.45, 163305.39 410568.49, 161861.51 41046

In [43]:
for de605_pointing in de605_pointings:
    print(f"- alt {de605_pointing.alt.deg:.1f}˚, az {de605_pointing.az.deg:.1f}˚")

- alt 53.1˚, az 333.6˚
- alt 44.9˚, az 322.9˚
- alt 37.7˚, az 318.8˚
